In [1]:
import os
import shutil

import re
import time
import pandas as pd

#encoding="utf8"

csv_fileScopus = pd.read_csv('19-07-21-Scopus/FinalSnowballing/All_CleanSnowballs.vf.csv', header=0, engine='python', warn_bad_lines=True, error_bad_lines=False)


csv_fileScopus

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Link,Abstract,Author Keywords,Index Keywords,References,Document Type,Publication Stage,Open Access,Source,EID
0,"Bengong Y., Haomin J.",56584666200;57222136866;,Semi-supervised method for text classification...,2020,Data Analysis and Knowledge Discovery,4,10,NaN,58,69,...,https://www.scopus.com/inward/record.uri?eid=2...,[Objective] This paper proposes a new semi-sup...,Classifier Divergence; Ensemble Learning; Samp...,NaN,"Li, M, Zhou, Z H., Improve Computer-Aided Diag...",Article,Final,NaN,Scopus,2-s2.0-85101638051
1,"Vak D., Hwang K., Faulks A., Jung Y.-S., Clark...",6507210181;56468591200;56527232500;55955261500...,3D printer based slot-die coater as a lab-to-f...,2015,Advanced Energy Materials,5,4,1401539,NaN,NaN,...,https://www.scopus.com/inward/record.uri?eid=2...,A 3D printer based slot-die coater is develope...,additive manufacturing; organic solar cells; p...,3D printers; Perovskite; Printing machinery; P...,"Graetzel, M., Janssen, R.A.J., Mitzi, D.B., Sa...",Article,Final,NaN,Scopus,2-s2.0-84923365779
2,"Brierley C., Sawalha M., Atwell E.",35179028300;52164546400;26631780000;,Open-source boundary-annotated corpus for Arab...,2012,Proceedings of the 8th International Conferenc...,NaN,NaN,NaN,1011,1016,...,https://www.scopus.com/inward/record.uri?eid=2...,A boundary-annotated and part-of-speech tagged...,Phrase break prediction; Prosodic annotation; ...,Forecasting; Annotation scheme; Computational ...,"Ali, A.Y., (2000) The Holy Quran (Translated b...",Conference Paper,Final,NaN,Scopus,2-s2.0-84991925390
3,"Bengio Y., Schwenk H., Senécal J.-S., Morin F....",7003958245;7005072756;36861992700;57197335675;...,Neural probabilistic language models,2006,Studies in Fuzziness and Soft Computing,194,NaN,NaN,137,186,...,https://www.scopus.com/inward/record.uri?eid=2...,A central goal of statistical language modelin...,NaN,NaN,https://sourceforge.net/projects/math-atlas/at...,Article,Final,NaN,Scopus,2-s2.0-33845260073
4,"Banerjee K., Mandal C., Sarkar D.",55321714500;56032490800;56263537600;,A translation validation framework for symboli...,2015,2015 IEEE 15th International Working Conferenc...,NaN,NaN,7335421,247,252,...,https://www.scopus.com/inward/record.uri?eid=2...,A compiler is a computer program which transla...,code optimization; equivalence checking; Finit...,Codes (symbols); Computer programming language...,"Pnueli, A., Siegel, M., Singerman, E., Transla...",Conference Paper,Final,NaN,Scopus,2-s2.0-84963568382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,"Ren S., Zhang Z., Liu S., Zhou M., Ma S.",57208219575;57202958159;56181265100;5558789080...,Unsupervised neural machine translation with S...,2019,33rd AAAI Conference on Artificial Intelligenc...,NaN,NaN,NaN,241,248,...,https://www.scopus.com/inward/record.uri?eid=2...,"Without real bilingual corpus available, unsup...",NaN,Computational linguistics; Computer aided lang...,"Artetxe, M., Labaka, G., Agirre, E., Cho, K., ...",Conference Paper,Final,NaN,Scopus,2-s2.0-85090177490
1327,"Li M., Zong C., Ng H.T.",57199161283;7005615574;47061923200;,Automatic evaluation of Chinese translation ou...,2011,ACL-HLT 2011 - Proceedings of the 49th Annual ...,2,NaN,NaN,159,164,...,https://www.scopus.com/inward/record.uri?eid=2...,Word is usually adopted as the smallest unit i...,NaN,Automatic evaluation; Chinese language; Human ...,"Banerjee, S., Lavie, A., METEOR: An automatic ...",Conference Paper,Final,NaN,Scopus,2-s2.0-84859072090
1328,"Dyer C., Muresan S., Resnik P.",51664869700;57206143958;6602712741;,Generalizing word lattice translation,2008,ACL-08: HLT - 46th Annual Meeting of the Assoc...,NaN,NaN,NaN,1012,1020,...,https://www.scopus.com/inward/record.uri?eid=2...,Word lattice decoding has proven useful in spo...,NaN,Context-free; Finite state; Lattice decoding; ...,"Bangalore, S., Riccardi, G., Finite state mode...",Conference Pap

In [2]:
csv_fileScopus.dropna(subset = ["Author Keywords"], inplace=True)
csv_fileScopus.dropna(subset = ["Title"], inplace=True)
csv_fileScopus.reset_index(drop=True, inplace=True)
len(csv_fileScopus)

639

In [3]:
len(csv_fileScopus)

639

In [4]:
import string
csv_fileScopus['CleanTitles'] = csv_fileScopus['Title'].str.lower()
csv_fileScopus['CleanAuthKeys'] = csv_fileScopus['Author Keywords'].str.lower()
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = str(text).replace(punctuation, ' ')
    return text

csv_fileScopus['CleanTitles'] = csv_fileScopus['CleanTitles'].apply(remove_punctuations)
csv_fileScopus['CleanAuthKeys'] = csv_fileScopus['CleanAuthKeys'].apply(remove_punctuations)

len(csv_fileScopus)

639

In [5]:
PatternsInn = []
#Remove All words that could mess our classification
Words1 = ['machine','automatic']
Words2 = ['translation','translator','translators','translating','translate','translated']
#Lang = [word.lower() for word in Languages]
# Recognition for IEEE
for i in range(len(csv_fileScopus["CleanTitles"])): 
    #res = re.sub(r'[^\w\s]', ' ', df.astype(str)["Document Title"][i]+df.astype(str)["Abstract"][i]+df.astype(str)["Author Keywords"][i])
    #res = re.sub(r'[^\w\s]', ' ', str(csv_fileIEEE["Titles & Keywords"][i]))
    #texto = res.lower().split()
    decision = 0 
    for j in range(len(Words2)):
        a = re.sub(r'[^\w\s]', ' ', str(csv_fileScopus["CleanTitles"][i]))
        b1 = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Words1[0].split(' ')) + r'\b'
        b2 = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Words1[1].split(' ')) + r'\b'
        b = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Words2[j].split(' ')) + r'\b'
        c = bool((re.search(b,a) and re.search(b1,a)) or (re.search(b,a) and re.search(b2,a)) or (re.search('mt',a)) or (re.search('smt',a)) or (re.search('nmt',a)) or (re.search('rbmt',a)) or (re.search('hmt',a)))
        if c == True:
            decision = 1
            break
    PatternsInn.append(decision)
    
csv_fileScopus["TitlesPatterns"] = PatternsInn
print("The number of Patterns that the program can recognize :"+str(len(Words2) * len(Words1)))
csv_fileScopus

The number of Patterns that the program can recognize :12


,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Index Keywords,References,Document Type,Publication Stage,Open Access,Source,EID,CleanTitles,CleanAuthKeys,TitlesPatterns
0,"Bengong Y., Haomin J.",56584666200;57222136866;,Semi-supervised method for text classification...,2020,Data Analysis and Knowledge Discovery,4,10,NaN,58,69,...,NaN,"Li, M, Zhou, Z H., Improve Computer-Aided Diag...",Article,Final,NaN,Scopus,2-s2.0-85101638051,semi supervised method for text classification...,classifier divergence ensemble learning samp...,0
1,"Vak D., Hwang K., Faulks A., Jung Y.-S., Clark...",6507210181;56468591200;56527232500;55955261500...,3D printer based slot-die coater as a lab-to-f...,2015,Advanced Energy Materials,5,4,1401539,NaN,NaN,...,3D printers; Perovskite; Printing machinery; P...,"Graetzel, M., Janssen, R.A.J., Mitzi, D.B., Sa...",Article,Final,NaN,Scopus,2-s2.0-84923365779,3d printer based slot die coater as a lab to f...,additive manufacturing organic solar cells p...,0
2,"Brierley C., Sawalha M., Atwell E.",35179028300;52164546400;26631780000;,Open-source boundary-annotated corpus for Arab...,2012,Proceedings of the 8th International Conferenc...,NaN,NaN,NaN,1011,1016,...,Forecasting; Annotation scheme; Computational ...,"Ali, A.Y., (2000) The Holy Quran (Translated b...",Conference Paper,Final,NaN,Scopus,2-s2.0-84991925390,open source boundary annotated corpus for arab...,phrase break prediction prosodic annotation ...,0
3,"Banerjee K., Mandal C., Sarkar D.",55321714500;56032490800;56263537600;,A translation validation framework for symboli...,2015,2015 IEEE 15th International Working Conferenc...,NaN,NaN,7335421,247,252,...,Codes (symbols); Computer programming language...,"Pnueli, A., Siegel, M., Singerman, E., Transla...",Conference Paper,Final,NaN,Scopus,2-s2.0-84963568382,a translation validation framework for symboli...,code optimization equivalence checking finit...,0
4,"Raybaud S., Lavecchia C., Langlois D., Smaïli K.",24759531900;24344121600;35842911600;6506309427;,New confidence measures for statistical machin...,2009,ICAART 2009 - Proceedings of the 1st Internati...,NaN,NaN,NaN,61,68,...,Confidence measure; Linguistic features langua...,"Akiba, Y., Sumita, E., Nakaiwa, H., Yamamoto, ...",Conference Paper,Final,NaN,Scopus,2-s2.0-70349445586,new confidence measures for statistical machin...,confidence measure linguistic features langua...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,Xia Y.,57210580691;,Research on statistical machine translation mo...,2020,Computing,102,3,NaN,643,661,...,Alignment; Computational linguistics; Computer...,"Claycomb, J., Abreu-Goodger, C., Buck, A.H., R...",Article,Final,NaN,Scopus,2-s2.0-85071025747,research on statistical machine translation mo...,deep neural network machine translation mode...,1
635,"Lee H.-Y., Tseng B.-H., Wen T.-H., Tsao Y.",34969292900;57188981571;56900572400;13608047100;,Personalizing Recurrent-Neural-Network-Based L...,2017,IEEE/ACM Transactions on Audio Speech and Lang...,25,3,NaN,519,530,...,Computational linguistics; Mobile devices; Mod...,"Speretta, M., Gauch, S., Personalized search b...",Article,Final,NaN,Scopus,2-s2.0-85014901162,personalizing recurrent neural network based l...,personalized language modeling recurrent neur...,0
636,Al-Khalifa H.S.,16241119700;,Introducing Arabic sign language for mobile ph...,2010,Lecture Notes in Computer Science (including s...,6180 LNCS,PART 2,NaN,213,220,...,Arabic languages; Arabic sign language; Arabic...,"Al-Showaier, H., Standardizing Arabic sign lan...",Conference Paper,Final,NaN,Scopus,2-s2.0-77955066477,introducing arabic sign language for mobile ph...,arabic language hearing impaired mobile phon...,0
637,"Prithvi P., Kumar T.K.",57211540900;35292229000;,Enhanced Beam-Search Restructuring Restraint f...,2019,Proceedings of 2019 3rd IEEE International Con...,NaN,NaN,8869367,NaN,NaN,...,Computational linguistics; Pol

In [6]:
PatternsInn = []
#Remove All words that could mess our classification
Words1 = ['machine','automatic']
Words2 = ['translation','translator','translating','translate','translated']
#Lang = [word.lower() for word in Languages]
# Recognition for IEEE
for i in range(len(csv_fileScopus["CleanAuthKeys"])): 
    #res = re.sub(r'[^\w\s]', ' ', df.astype(str)["Document Title"][i]+df.astype(str)["Abstract"][i]+df.astype(str)["Author Keywords"][i])
    #res = re.sub(r'[^\w\s]', ' ', str(csv_fileIEEE["Titles & Keywords"][i]))
    #texto = res.lower().split()
    decision = 0 
    for j in range(len(Words2)):
        a = re.sub(r'[^\w\s]', ' ', str(csv_fileScopus["CleanAuthKeys"][i]))
        b1 = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Words1[0].split(' ')) + r'\b'
        b2 = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Words1[1].split(' ')) + r'\b'
        b = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Words2[j].split(' ')) + r'\b'
        c = bool((re.search(b,a) and re.search(b1,a)) or (re.search(b,a) and re.search(b2,a)) or (re.search('mt',a)) or (re.search('smt',a)) or (re.search('nmt',a)) or (re.search('rbmt',a)) or (re.search('hmt',a)))
        if c == True:
            decision = 1
            break
    PatternsInn.append(decision)
    
csv_fileScopus["AuthKeysPatterns"] = PatternsInn
print("The number of Patterns that the program can recognize :"+str(len(Words2) * len(Words1)))
csv_fileScopus

The number of Patterns that the program can recognize :10


,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,References,Document Type,Publication Stage,Open Access,Source,EID,CleanTitles,CleanAuthKeys,TitlesPatterns,AuthKeysPatterns
0,"Bengong Y., Haomin J.",56584666200;57222136866;,Semi-supervised method for text classification...,2020,Data Analysis and Knowledge Discovery,4,10,NaN,58,69,...,"Li, M, Zhou, Z H., Improve Computer-Aided Diag...",Article,Final,NaN,Scopus,2-s2.0-85101638051,semi supervised method for text classification...,classifier divergence ensemble learning samp...,0,0
1,"Vak D., Hwang K., Faulks A., Jung Y.-S., Clark...",6507210181;56468591200;56527232500;55955261500...,3D printer based slot-die coater as a lab-to-f...,2015,Advanced Energy Materials,5,4,1401539,NaN,NaN,...,"Graetzel, M., Janssen, R.A.J., Mitzi, D.B., Sa...",Article,Final,NaN,Scopus,2-s2.0-84923365779,3d printer based slot die coater as a lab to f...,additive manufacturing organic solar cells p...,0,0
2,"Brierley C., Sawalha M., Atwell E.",35179028300;52164546400;26631780000;,Open-source boundary-annotated corpus for Arab...,2012,Proceedings of the 8th International Conferenc...,NaN,NaN,NaN,1011,1016,...,"Ali, A.Y., (2000) The Holy Quran (Translated b...",Conference Paper,Final,NaN,Scopus,2-s2.0-84991925390,open source boundary annotated corpus for arab...,phrase break prediction prosodic annotation ...,0,0
3,"Banerjee K., Mandal C., Sarkar D.",55321714500;56032490800;56263537600;,A translation validation framework for symboli...,2015,2015 IEEE 15th International Working Conferenc...,NaN,NaN,7335421,247,252,...,"Pnueli, A., Siegel, M., Singerman, E., Transla...",Conference Paper,Final,NaN,Scopus,2-s2.0-84963568382,a translation validation framework for symboli...,code optimization equivalence checking finit...,0,1
4,"Raybaud S., Lavecchia C., Langlois D., Smaïli K.",24759531900;24344121600;35842911600;6506309427;,New confidence measures for statistical machin...,2009,ICAART 2009 - Proceedings of the 1st Internati...,NaN,NaN,NaN,61,68,...,"Akiba, Y., Sumita, E., Nakaiwa, H., Yamamoto, ...",Conference Paper,Final,NaN,Scopus,2-s2.0-70349445586,new confidence measures for statistical machin...,confidence measure linguistic features langua...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,Xia Y.,57210580691;,Research on statistical machine translation mo...,2020,Computing,102,3,NaN,643,661,...,"Claycomb, J., Abreu-Goodger, C., Buck, A.H., R...",Article,Final,NaN,Scopus,2-s2.0-85071025747,research on statistical machine translation mo...,deep neural network machine translation mode...,1,1
635,"Lee H.-Y., Tseng B.-H., Wen T.-H., Tsao Y.",34969292900;57188981571;56900572400;13608047100;,Personalizing Recurrent-Neural-Network-Based L...,2017,IEEE/ACM Transactions on Audio Speech and Lang...,25,3,NaN,519,530,...,"Speretta, M., Gauch, S., Personalized search b...",Article,Final,NaN,Scopus,2-s2.0-85014901162,personalizing recurrent neural network based l...,personalized language modeling recurrent neur...,0,0
636,Al-Khalifa H.S.,16241119700;,Introducing Arabic sign language for mobile ph...,2010,Lecture Notes in Computer Science (including s...,6180 LNCS,PART 2,NaN,213,220,...,"Al-Showaier, H., Standardizing Arabic sign lan...",Conference Paper,Final,NaN,Scopus,2-s2.0-77955066477,introducing arabic sign language for mobile ph...,arabic language hearing impaired mobile phon...,0,0
637,"Prithvi P., Kumar T.K.",57211540900;35292229000;,Enhanced Beam-Search Restructuring Restraint f...,2019,Proceedings of 2019 3rd IEEE International Con...,NaN,NaN,8869367,NaN,NaN,...,"Och, F.J., Ney, H., Discriminative training an...",Conference Paper,Final,NaN,Scopus,2-s2.0-85074345025,enhanced beam search restructuring restraint f...,beam search bilingual evaluation understudy ...,1,0


In [7]:
csv_fileScopus.to_excel("19-07-21-Scopus\FinalSnowballing\Scopus_MT_Indexed.xlsx", index=False)

C:\Users\HP\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:941: UserWarning: Ignoring URL 'http://www.latice.rnu.tn/websign,%20The%20homepage%20of%20WebSign.%20%5bOnline%5d.%20Available;%20El%20Ghoul,%20O.,%20Jemni,%20M.,%20Using%20virtual%20avatars%20to%20generate%20accessible%20courses%20for%20deaf%20(2013)%20Third%20International%20Conference%20on%20E-learning%20and%20Distance%20Learning,%20,%20Riyadh,%20Saudi%20Arabia,%204-7%20February;%20Jemni,%20M.,%20El%20Ghoul,%20O.,%20An%20avatar%20based%20approach%20for%20automatic%20interpretation%20of%20text%20to%20sign%20language%20(2007)%209th%20European%20Conference%20for%20the%20Advancement%20of%20the%20Assistive%20Technologies%20in%20Europe,%20,%20San%20Sebastián%20(Spain),%203-5%20October;%20Jemni,%20M.,%20Elghoul,%20O.,%20A%20system%20to%20make%20signs%20using%20collaborative%20approach%20(2008)%2011th%20International%20Conference%20ICCHP%202008,%20pp.%20670-677.%20,%20Linz,%20Austria,%20July%209-11,%202008.%20Proceedings.%20Lec

In [7]:
dataframeMT = csv_fileScopus.loc[(csv_fileScopus['TitlesPatterns'] == 1) | (csv_fileScopus['AuthKeysPatterns'] == 1)]
dataframeMT.reset_index(drop=True, inplace=True)
dataframeMT.to_excel("19-07-21-Scopus\FinalSnowballing\Scopus_only_MT.xlsx",index=False)
len(dataframeMT)

C:\Users\HP\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:941: UserWarning: Ignoring URL 'http://www.latice.rnu.tn/websign,%20The%20homepage%20of%20WebSign.%20%5bOnline%5d.%20Available;%20El%20Ghoul,%20O.,%20Jemni,%20M.,%20Using%20virtual%20avatars%20to%20generate%20accessible%20courses%20for%20deaf%20(2013)%20Third%20International%20Conference%20on%20E-learning%20and%20Distance%20Learning,%20,%20Riyadh,%20Saudi%20Arabia,%204-7%20February;%20Jemni,%20M.,%20El%20Ghoul,%20O.,%20An%20avatar%20based%20approach%20for%20automatic%20interpretation%20of%20text%20to%20sign%20language%20(2007)%209th%20European%20Conference%20for%20the%20Advancement%20of%20the%20Assistive%20Technologies%20in%20Europe,%20,%20San%20Sebastián%20(Spain),%203-5%20October;%20Jemni,%20M.,%20Elghoul,%20O.,%20A%20system%20to%20make%20signs%20using%20collaborative%20approach%20(2008)%2011th%20International%20Conference%20ICCHP%202008,%20pp.%20670-677.%20,%20Linz,%20Austria,%20July%209-11,%202008.%20Proceedings.%20Lec

328

In [8]:
dataframeMT['Titles & Keywords'] = dataframeMT['Title'] +' '+ dataframeMT['Author Keywords']
dataframeMT['Titles & Keywords'] = dataframeMT['Titles & Keywords'].str.lower()
len(dataframeMT)

<ipython-input-8-60b314376a67>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeMT['Titles & Keywords'] = dataframeMT['Title'] +' '+ dataframeMT['Author Keywords']
<ipython-input-8-60b314376a67>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeMT['Titles & Keywords'] = dataframeMT['Titles & Keywords'].str.lower()


328

In [9]:
PatternsInn = []
#Remove All words that could mess our classification
Patterns = ['image','3d','2d','sign language','signal','speech','spoken','transliteration','dialect','video','twitter','facebook','tweet','comment','social network','social networking','post translational','translation initiation','translational science','braille','binary translation','kinematic','unstructured data']
#Lang = [word.lower() for word in Languages]
# Recognition for IEEE
for i in range(len(dataframeMT["Titles & Keywords"])): 
    #res = re.sub(r'[^\w\s]', ' ', df.astype(str)["Document Title"][i]+df.astype(str)["Abstract"][i]+df.astype(str)["Author Keywords"][i])
    #res = re.sub(r'[^\w\s]', ' ', str(csv_fileIEEE["Titles & Keywords"][i]))
    #texto = res.lower().split()
    PatternsIn = []
    for j in range(len(Patterns)):
        a = re.sub(r'[^\w\s]', ' ', str(dataframeMT["Titles & Keywords"][i]))
        b = r'\b' + r'\b.*\b'.join(re.escape(word) for word in Patterns[j].split(' ')) + r'\b'
        c = bool(re.search(b,a))
        if c == True:
            PatternsIn.append(Patterns[j])
    PatternsInn.append(PatternsIn)
    
dataframeMT["Patterns"] = PatternsInn
print("The number of Patterns that the program can recognize :"+str(len(Patterns)))
dataframeMT

The number of Patterns that the program can recognize :23


<ipython-input-9-c87db1d55a74>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframeMT["Patterns"] = PatternsInn


,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Publication Stage,Open Access,Source,EID,CleanTitles,CleanAuthKeys,TitlesPatterns,AuthKeysPatterns,Titles & Keywords,Patterns
0,"Banerjee K., Mandal C., Sarkar D.",55321714500;56032490800;56263537600;,A translation validation framework for symboli...,2015,2015 IEEE 15th International Working Conferenc...,NaN,NaN,7335421,247,252,...,Final,NaN,Scopus,2-s2.0-84963568382,a translation validation framework for symboli...,code optimization equivalence checking finit...,0,1,a translation validation framework for symboli...,[]
1,"Raybaud S., Lavecchia C., Langlois D., Smaïli K.",24759531900;24344121600;35842911600;6506309427;,New confidence measures for statistical machin...,2009,ICAART 2009 - Proceedings of the 1st Internati...,NaN,NaN,NaN,61,68,...,Final,NaN,Scopus,2-s2.0-70349445586,new confidence measures for statistical machin...,confidence measure linguistic features langua...,1,1,new confidence measures for statistical machin...,[]
2,"Germann U., Jahr M., Knight K., Marcu D., Yama...",7005251202;55550576400;7202745471;6701627574;5...,Fast and optimal decoding for machine translation,2004,Artificial Intelligence,154,01-Feb,NaN,127,143,...,Final,"All Open Access, Green",Scopus,2-s2.0-1242310001,fast and optimal decoding for machine translation,decoding machine translation mt smt statis...,1,1,fast and optimal decoding for machine translat...,[]
3,"Cicekli I., Güvenir H.A.",6603079400;6603878863;,Learning translation templates from bilingual ...,2001,Applied Intelligence,15,1,NaN,57,76,...,Final,"All Open Access, Green",Scopus,2-s2.0-0035397561,learning translation templates from bilingual ...,corpus based machine translation example base...,0,1,learning translation templates from bilingual ...,[]
4,"Jiang H.-F., Li S., Fu G.-H., Zhao T.-J., Zhan...",55017668100;36063951100;7202721096;7402268377;...,Statistical machine translation model based on...,2009,Ruan Jian Xue Bao/Journal of Software,20,5,NaN,1241,1253,...,Final,NaN,Scopus,2-s2.0-65749110133,statistical machine translation model based on...,global reordering machine translation non is...,1,1,statistical machine translation model based on...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,"Xiao T., Zhu J.-B.",57206520287;8556252200;,On decoding with augmented hierarchical phrase...,2016,Jisuanji Xuebao/Chinese Journal of Computers,39,4,NaN,808,821,...,Final,NaN,Scopus,2-s2.0-84966670140,on decoding with augmented hierarchical phrase...,decoding hierarchical phrase based model mac...,0,1,on decoding with augmented hierarchical phrase...,[]
324,"Shen S.-Q., Liu Y., Sun M.-S.",57155609300;57196311263;7403180987;,Optimizing Non-Decomposable Evaluation Metrics...,2017,Journal of Computer Science and Technology,32,4,NaN,796,804,...,Final,NaN,Scopus,2-s2.0-85024097293,optimizing non decomposable evaluation metrics...,neural machine translation non decomposable e...,1,1,optimizing non-decomposable evaluation metrics...,[]
325,"Cao R., Freitas C., Chan L., Sun M., Jiang H.,...",55619927900;57196346535;35208005400;5645934120...,ProLanGO: Protein function prediction using ne...,2017,Molecules,22,10,1732,NaN,NaN,...,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85032702712,prolango protein function prediction using ne...,machine learning neural machine translation ...,1,1,prolango: protein function prediction using ne...,[]
326,Xia Y.,57210580691;,Research on statistical machine translation mo...,2020,Computing,102,3,NaN,643,661,...,Final,NaN,Scopus,2-s2.0-85071025747,research on statistical machine translation mo...,deep neural network machine translation mode...,1,1,research on statistical machine translation mo...,[]


In [10]:
dataframeMT.to_excel("19-07-21-Scopus\FinalSnowballing\Scopus_MT_final.xlsx",index=False)

C:\Users\HP\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:941: UserWarning: Ignoring URL 'http://www.latice.rnu.tn/websign,%20The%20homepage%20of%20WebSign.%20%5bOnline%5d.%20Available;%20El%20Ghoul,%20O.,%20Jemni,%20M.,%20Using%20virtual%20avatars%20to%20generate%20accessible%20courses%20for%20deaf%20(2013)%20Third%20International%20Conference%20on%20E-learning%20and%20Distance%20Learning,%20,%20Riyadh,%20Saudi%20Arabia,%204-7%20February;%20Jemni,%20M.,%20El%20Ghoul,%20O.,%20An%20avatar%20based%20approach%20for%20automatic%20interpretation%20of%20text%20to%20sign%20language%20(2007)%209th%20European%20Conference%20for%20the%20Advancement%20of%20the%20Assistive%20Technologies%20in%20Europe,%20,%20San%20Sebastián%20(Spain),%203-5%20October;%20Jemni,%20M.,%20Elghoul,%20O.,%20A%20system%20to%20make%20signs%20using%20collaborative%20approach%20(2008)%2011th%20International%20Conference%20ICCHP%202008,%20pp.%20670-677.%20,%20Linz,%20Austria,%20July%209-11,%202008.%20Proceedings.%20Lec